# 가중치만 적용 버전

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
try:
    df = pd.read_csv("D:/semi2/MH/df/질환별/이상지질혈증-df.csv", encoding='cp949')
except:
    df = pd.read_csv("D:/semi2/MH/df/질환별/이상지질혈증-df.csv", encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DY' # 예시: 고혈압
blocks = {
    'Model 1': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'LQ_LF_S', 'EQ5D'],
    'Model 2': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 3': ['C(HE_obe)', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)']
}

# 유의성 표시 함수
def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (*** < 0.001)"
    elif p < 0.01: return f"{p:.3f} (** < 0.01)"
    elif p < 0.05: return f"{p:.3f} (* < 0.05)"
    else: return f"{p:.3f}"

final_results = []
cumulative_features = []

for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    # 결측치 제거 (가중치 포함)
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex'] + clean_vars)
    
    # 모델 피팅 (가중치 적용 핵심 부분)
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    result = model.fit()
    
    # 결과 수집
    for var in X.columns:
        if var == 'Intercept': continue
        
        # 변수명 및 카테고리 추출
        if '[T.' in var:
            v_main = var.split('[')[0].replace('C(', '').replace(')', '')
            category = var.split('[T.')[1].replace(']', '')
        else:
            v_main = var
            category = 'Continuous'
            
        or_val = np.exp(result.params[var])
        ci = np.exp(result.conf_int().loc[var])
        p_val = result.pvalues[var]
        
        final_results.append({
            '변수명': v_main,
            '카테고리': category,
            '모델': model_name,
            'OR (95% CI)': f"{or_val:.2f} ({ci[0]:.2f} - {ci[1]:.2f})",
            'p-value': get_p_display(p_val)
        })

# 3. REF 행 추가 및 최종 정리
final_df_list = []
df_res = pd.DataFrame(final_results)

for model in [f'Model {i}' for i in range(1, 6)]:
    model_subset = df_res[df_res['모델'] == model]
    unique_vars = model_subset['변수명'].unique()
    
    for v in unique_vars:
        v_rows = model_subset[model_subset['변수명'] == v]
        # 범주형 변수일 경우 최상단에 REF 추가
        if v_rows.iloc[0]['카테고리'] != 'Continuous':
            final_df_list.append({'변수명': v, '카테고리': 'REF_Category', '모델': model, 'OR (95% CI)': 'REF', 'p-value': '-'})
        
        for _, row in v_rows.iterrows():
            final_df_list.append(row)

# 4. 엑셀 저장
final_output = pd.DataFrame(final_df_list)
final_output.to_excel(f'Table3_Final_Weighted_{target_col}.xlsx', index=False)
print("엑셀 파일 저장이 완료되었습니다.")

엑셀 파일 저장이 완료되었습니다.


# 가중치/충화/군집 적용 버전

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
file_name = "D:/semi2/MH/df/질환별/이상지질혈증-df.csv"
try:
    df = pd.read_csv(file_name, encoding='cp949')
except:
    df = pd.read_csv(file_name, encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DY'  
blocks = {
    'Model 1': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'EQ5D'],
    'Model 2': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 3': ['C(HE_obe)','HE_ht', 'HE_wt', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)']
}

def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (***)"
    elif p < 0.01: return f"{p:.3f} (**)"
    elif p < 0.05: return f"{p:.3f} (*)"
    else: return f"{p:.3f}"

final_results = []
cumulative_features = []

for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    # 필수 변수 리스트 (타겟, 가중치, PSU, 독립변수들)
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex', 'psu'] + clean_vars).copy()
    
    # 매트릭스 생성
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    
    # [핵심] 복합표본 대안 분석법
    # var_weights로 가중치 적용, cov_type='cluster'와 groups='psu'로 군집/층화 효과 반영
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    
    try:
        # 분산 추정 시 군집(PSU)을 고려하여 표준오차 보정
        result = model.fit(cov_type='cluster', cov_kwds={'groups': temp_df['psu']})
        
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        for var in X.columns:
            if var == 'Intercept': continue
            
            # 카테고리 추출
            if '[T.' in var:
                v_main = var.split('[')[0].replace('C(', '').replace(')', '')
                category = var.split('[T.')[1].replace(']', '')
            else:
                v_main = var
                category = 'Continuous'
                
            or_val = np.exp(params[var])
            ci_low, ci_high = np.exp(conf.loc[var])
            
            final_results.append({
                '변수명': v_main, '카테고리': category, '모델': model_name,
                'OR (95% CI)': f"{or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})",
                'p-value': get_p_display(pvals[var])
            })
    except Exception as e:
        print(f"{model_name} 분석 중 오류 발생: {e}")

# 3. REF 행 추가 및 저장 (이전 로직과 동일)
final_df_list = []
df_res = pd.DataFrame(final_results)

for model in [f'Model {i}' for i in range(1, 6)]:
    m_sub = df_res[df_res['모델'] == model]
    for v in m_sub['변수명'].unique():
        v_rows = m_sub[m_sub['변수명'] == v]
        if v_rows.iloc[0]['카테고리'] != 'Continuous':
            final_df_list.append({'변수명': v, '카테고리': 'REF', '모델': model, 'OR (95% CI)': 'REF', 'p-value': '-'})
        final_df_list.extend(v_rows.to_dict('records'))

pd.DataFrame(final_df_list).to_excel(f'Table3_이상지질혈증_Analysis_{target_col}.xlsx', index=False)
print("분석이 완료되었습니다.")


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


분석이 완료되었습니다.


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
